In [51]:
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToJson
import urllib.request
import time
import pytz
import json
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

def scheduleDateTime(stop_time):
    dt=date.today()
    time_obj = datetime.strptime(stop_time, "%H:%M:%S").time()
    dt_with_time = datetime.combine(dt, time_obj)
    dt_with_time = pytz.timezone('Europe/Warsaw').localize(dt_with_time)
    return dt_with_time

feed = gtfs_realtime_pb2.FeedMessage()
response = urllib.request.urlopen('https://mkuran.pl/gtfs/wkd.pb')
feed.ParseFromString(response.read())

rozklad = pd.read_csv(r'C:\Users\hmackowski\Desktop\python scripts\wkd\wkd_gtfs\stop_times.txt', sep=',')
print(rozklad)
json_wkd = MessageToJson(feed)
#print(json_wkd)
json_wkd=json.loads(json_wkd)
json_wkd=json_wkd['entity']
#print(json_wkd)



     trip_id arrival_time departure_time stop_id  stop_sequence
0      C-303     05:05:00       05:05:00   wsrod              0
1      C-303     05:07:00       05:07:00   wocho              1
2      C-303     05:10:00       05:10:00   wzach              2
3      C-303     05:12:00       05:12:00   wreor              3
4      C-303     05:14:00       05:14:00   walje              4
...      ...          ...            ...     ...            ...
6283   D-101     25:12:00       25:12:00   brzoz             21
6284   D-101     25:14:00       25:14:00   gmokr             22
6285   D-101     25:16:00       25:16:00   gmpia             23
6286   D-101     25:18:00       25:18:00   gmjor             24
6287   D-101     25:23:00       25:23:00   gmrad             25

[6288 rows x 5 columns]


opis

In [52]:
df = pd.json_normalize(json_wkd)


In [53]:
df = pd.json_normalize(json_wkd, record_path=['tripUpdate','stopTimeUpdate'],meta=['id'])
tz = pytz.timezone('Europe/Warsaw')
df['departure.time'] = pd.to_datetime(df['departure.time'], unit='s').apply(lambda x: pd.Timestamp(x).tz_localize(None).tz_localize('UTC').tz_convert(tz))
print(df)

    stopId            departure.time      id
0    gmokr 2023-03-29 14:18:00+02:00  D-3004
1    gmrad 2023-03-29 14:05:00+02:00  D-3004
2    gmpia 2023-03-29 14:16:00+02:00  D-3004
3    kazim 2023-03-29 14:29:00+02:00  D-3004
4    plzac 2023-03-29 14:32:00+02:00  D-3004
..     ...                       ...     ...
275  walje 2023-03-29 13:14:00+02:00   D-393
276  wsrod 2023-03-29 13:05:00+02:00   D-393
277  wzach 2023-03-29 13:10:00+02:00   D-393
278  wreor 2023-03-29 13:12:00+02:00   D-393
279  wocho 2023-03-29 13:07:00+02:00   D-393

[280 rows x 3 columns]


In [54]:
actual = df[df['stopId'] == 'nwwar']
  
print(actual)

rozklad_filtr = rozklad[rozklad['stop_id'] == 'nwwar']
  
print(rozklad_filtr)

    stopId            departure.time     id
36   nwwar 2023-03-29 12:53:00+02:00  D-378
65   nwwar 2023-03-29 12:37:00+02:00  D-381
93   nwwar 2023-03-29 12:23:00+02:00  D-370
120  nwwar 2023-03-29 12:38:00+02:00  D-374
137  nwwar 2023-03-29 13:07:00+02:00  D-385
161  nwwar 2023-03-29 12:24:00+02:00  D-377
179  nwwar 2023-03-29 12:08:00+02:00  D-368
226  nwwar 2023-03-29 13:08:00+02:00  D-380
245  nwwar 2023-03-29 12:07:00+02:00  D-373
256  nwwar 2023-03-29 12:52:00+02:00  D-383
     trip_id arrival_time departure_time stop_id  stop_sequence
14     C-303     05:37:00       05:37:00   nwwar             14
36     C-105     05:52:00       05:52:00   nwwar             14
62     C-309     06:12:00       06:12:00   nwwar             14
84     C-313     06:32:00       06:32:00   nwwar             14
103    C-109     06:52:00       06:52:00   nwwar             14
...      ...          ...            ...     ...            ...
6176   D-153     23:14:00       23:14:00   nwwar             14
6202

In [61]:
# merge dataframes based on key_col1 and key_col2
merged_df = pd.merge(actual, rozklad, left_on=['id','stopId'], right_on=['trip_id','stop_id'], how='left')

merged_df = merged_df.drop(columns=['trip_id','stop_id','departure_time'])
merged_df = merged_df.sort_values(by='departure.time')

merged_df['arrival_time'] = merged_df['arrival_time'].fillna(merged_df['departure.time'].apply(lambda x: x.strftime('%H:%M:%S')))
#print(merged_df)

merged_df['arrival_time'] = merged_df['arrival_time'].apply(scheduleDateTime)

merged_df['delay']=merged_df['departure.time']-merged_df['arrival_time']

merged_df['delay'] = merged_df['delay'].apply(lambda x: x.total_seconds() / 60)


merged_df['direction'] = merged_df['id'].apply(lambda x: 'W' if int(x[2:]) % 2 == 0 else 'P')
merged_df=merged_df[['direction','id','stopId','delay','arrival_time','departure.time']]
mask = merged_df['direction'] == 'P'
print(merged_df[mask].to_string(index=False))

mask = merged_df['direction'] == 'W'
print(merged_df[mask].to_string(index=False))

direction    id stopId  delay              arrival_time            departure.time
        P D-373  nwwar    0.0 2023-03-29 12:07:00+02:00 2023-03-29 12:07:00+02:00
        P D-377  nwwar    2.0 2023-03-29 12:22:00+02:00 2023-03-29 12:24:00+02:00
        P D-381  nwwar    0.0 2023-03-29 12:37:00+02:00 2023-03-29 12:37:00+02:00
        P D-383  nwwar    0.0 2023-03-29 12:52:00+02:00 2023-03-29 12:52:00+02:00
        P D-385  nwwar    0.0 2023-03-29 13:07:00+02:00 2023-03-29 13:07:00+02:00
direction    id stopId  delay              arrival_time            departure.time
        W D-368  nwwar    0.0 2023-03-29 12:08:00+02:00 2023-03-29 12:08:00+02:00
        W D-370  nwwar    0.0 2023-03-29 12:23:00+02:00 2023-03-29 12:23:00+02:00
        W D-374  nwwar    0.0 2023-03-29 12:38:00+02:00 2023-03-29 12:38:00+02:00
        W D-378  nwwar    0.0 2023-03-29 12:53:00+02:00 2023-03-29 12:53:00+02:00
        W D-380  nwwar    0.0 2023-03-29 13:08:00+02:00 2023-03-29 13:08:00+02:00
